# Q4 — Train & Evaluate a Keras-Based Classifier
Per rubric: use os.walk to collect filenames, create validation_generator, build `test_model` with 4 Conv2D and 5 Dense layers, checkpoint on val_accuracy, plot losses.

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

dataset_path = 'images_dataSAT'
# 1) os.walk to build fnames
fnames = []
for root, dirs, files in os.walk(dataset_path):
    for f in files:
        if f.lower().endswith(('.png','.jpg','.jpeg')):
            fnames.append(os.path.join(root,f))
print('Total files via os.walk:', len(fnames))

# generators with validation split
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_gen = datagen.flow_from_directory(dataset_path, target_size=(64,64), batch_size=4, subset='training', class_mode='categorical')
validation_generator = datagen.flow_from_directory(dataset_path, target_size=(64,64), batch_size=4, subset='validation', class_mode='categorical', shuffle=False)

# test_model: 4 Conv2D + 5 Dense layers
test_model = Sequential([
    Conv2D(16,3,activation='relu', input_shape=(64,64,3)),
    MaxPooling2D(),
    Conv2D(32,3,activation='relu'),
    MaxPooling2D(),
    Conv2D(64,3,activation='relu'),
    MaxPooling2D(),
    Conv2D(64,3,activation='relu'),  # 4th Conv2D
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),  # 1
    Dense(64, activation='relu'),   # 2
    Dense(32, activation='relu'),   # 3
    Dense(16, activation='relu'),   # 4
    Dense(train_gen.num_classes, activation='softmax')  # 5 output dense
])
print('Total layers in test_model:', len(test_model.layers))
test_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# checkpoint
ckpt = ModelCheckpoint('q4_best.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
es = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

hist = test_model.fit(train_gen, validation_data=validation_generator, epochs=3, callbacks=[ckpt, es])

# plot training & validation loss
plt.plot(hist.history['loss'], label='train_loss')
plt.plot(hist.history['val_loss'], label='val_loss')
plt.legend(); plt.title('Train vs Val Loss'); plt.xlabel('Epoch'); plt.show()